In [2]:
import pandas as pd

In [19]:
variants = ['15_mins', '30_mins', '1_hour', '3_hour', '24_hour']

In [23]:
cascade_lstm = pd.read_csv('../results/experiment_1.csv')
(cascade_lstm.groupby('variant')
.agg({'val_loss':'mean', 'val_mae':'mean'})
.stack()
.rename(index={'val_loss':'mse', 'val_mae':'mae'})
.to_frame().transpose()[variants]
.to_latex())

'\\begin{tabular}{lrrrrrrrrrr}\n\\toprule\nvariant & \\multicolumn{2}{l}{15\\_mins} & \\multicolumn{2}{l}{30\\_mins} & \\multicolumn{2}{l}{1\\_hour} & \\multicolumn{2}{l}{3\\_hour} & \\multicolumn{2}{l}{24\\_hour} \\\\\n{} &      mse &       mae &       mse &       mae &       mse &       mae &       mse &       mae &       mse &      mae \\\\\n\\midrule\n0 &  0.51204 &  0.556808 &  0.551627 &  0.559724 &  0.368663 &  0.493551 &  0.387819 &  0.477967 &  0.245614 &  0.34316 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [ ]:
res = pd.DataFrame